In [3]:
#IMPORTS

import os
import json
import geopandas as gpd
import numpy as numpy
import pandas as pd
import seaborn as sns

# Setting up barrios_gdf for cleaning

In this section, we read barrios_carcteristicas.geojson from datos/datos_out and we transform it into a dataframe using geopandas library

In [4]:
#SETTING UP barrios_caracteristicas as GDF

# SETTING UP BARRIOS DF
path = '../datos/datos_out/barrios_caracteristicas_final.geojson'


#CARGAMOS LOS DATOS DE LOS BARRIOS
with open(path) as json_file:
    json_data = json.load(json_file)

barrios_json = []
for i in range(len(json_data['features'])):
    barrios_json.append(json_data['features'][i])           #Guardamos resto de campos de geojson


barrios_gdf = gpd.GeoDataFrame.from_features(barrios_json)
barrios_gdf.crs = 'epsg:4326' #Aseguramos que la proyección es la adecuada para coordenadas GPS

barrios_gdf

,geometry,coddistrit,gis_gis_barrios_area,object_id_barrio,linkid,codbarrio,coddistbar,geo_point_2d,nombre_barrio,last_edited_user,...,num_colegios,id_caract_num_colegios,num_chargestations,id_caract_num_chargestations,pm25,id_caract_pm25,num_contenedores,id_caract_num_contenedores,num_transporte,id_caract_num_transporte
0,"POLYGON ((-0.33459 39.45478, -0.33326 39.45487...",11,917112.56250,62,0,5,115,"[39.44628964870906, -0.3326600366971329]",NATZARET,None,...,6.0,2,NaN,8,10.0,5,184.0,7,11.0,1
1,"POLYGON ((-0.38124 39.45463, -0.38281 39.44951...",9,374887.53125,63,0,3,093,"[39.45082750748332, -0.3853982961775011]",LA CREU COBERTA,None,...,1.0,2,2.0,8,21.0,5,162.0,7,8.0,1
2,"POLYGON ((-0.34709 39.47548, -0.34144 39.47379...",13,NaN,76,0,2,132,"[39.47198431553791, -0.3450427029615464]",CIUTAT JARDI,None,...,5.0,2,NaN,8,12.0,5,238.0,7,10.0,1
3,"POLYGON ((-0.28767 39.55682, -0.28829 39.55654...",17,NaN,601,18,5,175,"[39.55880504953868, -0.3031699565111558]",RAFALELL-VISTABELLA,None,...,NaN,2,NaN,8,12.0,5,NaN,7,NaN,1
4,"POLYGON ((-0.33151 39.48561, -0.33254 39.48046...",13,NaN,137,0,5,135,"[39.48035805242704, -0.3410905938386986]",LA CARRASCA,None,...,10.0,2,NaN,8,12.0,5,237.0,7,21.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"POLYGON ((-0.39047 39.47825, -0.38996 39.47754...",3,496668.75000,28,0,3,033,"[39.473565513491664, -0.39098170775459257]",LA PETXINA,None,...,11.0,2,NaN,8,12.0,5,279.0,7,14.0,1
84,"POLYGON ((-0.37567 39.47381, -0.37423 39.47351...",1,438658.31250,37,0,6,016,"[39.47010643870186, -0.3760489821791233]",SANT FRANCESC,None,...,2.0,2,2.0,8,12.0,5,580.0,7,27.0,1
85,"POLYGON ((-0.35018 39.42223, -0.35011 39.42227...",19,NaN,249,0,3,193,"[39.406290326215334, -0.33967854582733203]",PINEDO,None,...,2.0,2,NaN,8,10.0,5,185.0,7,14.0,1
86,"POLYGON ((-0.36720 39.50425, -0.36767 39.50338...",15,NaN,201,0,1,151,"[39.494312122911424, -0.3699310988383476]",ELS ORRIOLS,None,...,9.0,2,NaN,8,NaN,5,214.0,7,21.0,1


In [8]:
# CHECKING HOW MANY NULL VALUES ARE IN MY DF
barrios_gdf.isnull().sum()

geometry                         0
coddistrit                       0
gis_gis_barrios_area            32
object_id_barrio                 0
linkid                           0
codbarrio                        0
coddistbar                       0
geo_point_2d                     0
nombre_barrio                    0
last_edited_user                87
last_edited_date                87
%_zona_verde                    12
id_caract_%_zona_verde           0
nivel_acustico                   0
id_caract_nivel_acustico         0
num_hospitales                  40
id_caract_num_hospitales         0
num_colegios                     8
id_caract_num_colegios           0
num_chargestations              73
id_caract_num_chargestations     0
pm25                            28
id_caract_pm25                   0
num_contenedores                 1
id_caract_num_contenedores       0
num_transporte                   3
id_caract_num_transporte         0
dtype: int64

# Dropping columns we don't want
Attending to last_edited_user and last_edited_date we see that there are 119 missing values, which are for the whole column of the dataframe.
We are going to drop them using .drop() method

In [9]:
# DROPPING FULL NULL COLUMNS

barrios_gdf = barrios_gdf.drop(['last_edited_user', 'last_edited_date'], axis = 1)
barrios_gdf

,geometry,coddistrit,gis_gis_barrios_area,object_id_barrio,linkid,codbarrio,coddistbar,geo_point_2d,nombre_barrio,%_zona_verde,...,num_colegios,id_caract_num_colegios,num_chargestations,id_caract_num_chargestations,pm25,id_caract_pm25,num_contenedores,id_caract_num_contenedores,num_transporte,id_caract_num_transporte
0,"POLYGON ((-0.33459 39.45478, -0.33326 39.45487...",11,917112.56250,62,0,5,115,"[39.44628964870906, -0.3326600366971329]",NATZARET,0.118869,...,6.0,2,NaN,8,10.0,5,184.0,7,11.0,1
1,"POLYGON ((-0.38124 39.45463, -0.38281 39.44951...",9,374887.53125,63,0,3,093,"[39.45082750748332, -0.3853982961775011]",LA CREU COBERTA,0.037566,...,1.0,2,2.0,8,21.0,5,162.0,7,8.0,1
2,"POLYGON ((-0.34709 39.47548, -0.34144 39.47379...",13,NaN,76,0,2,132,"[39.47198431553791, -0.3450427029615464]",CIUTAT JARDI,0.205871,...,5.0,2,NaN,8,12.0,5,238.0,7,10.0,1
3,"POLYGON ((-0.28767 39.55682, -0.28829 39.55654...",17,NaN,601,18,5,175,"[39.55880504953868, -0.3031699565111558]",RAFALELL-VISTABELLA,NaN,...,NaN,2,NaN,8,12.0,5,NaN,7,NaN,1
4,"POLYGON ((-0.33151 39.48561, -0.33254 39.48046...",13,NaN,137,0,5,135,"[39.48035805242704, -0.3410905938386986]",LA CARRASCA,0.187377,...,10.0,2,NaN,8,12.0,5,237.0,7,21.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"POLYGON ((-0.39047 39.47825, -0.38996 39.47754...",3,496668.75000,28,0,3,033,"[39.473565513491664, -0.39098170775459257]",LA PETXINA,0.087284,...,11.0,2,NaN,8,12.0,5,279.0,7,14.0,1
84,"POLYGON ((-0.37567 39.47381, -0.37423 39.47351...",1,438658.31250,37,0,6,016,"[39.47010643870186, -0.3760489821791233]",SANT FRANCESC,0.134529,...,2.0,2,2.0,8,12.0,5,580.0,7,27.0,1
85,"POLYGON ((-0.35018 39.42223, -0.35011 39.42227...",19,NaN,249,0,3,193,"[39.406290326215334, -0.33967854582733203]",PINEDO,NaN,...,2.0,2,NaN,8,10.0,5,185.0,7,14.0,1
86,"POLYGON ((-0.36720 39.50425, -0.36767 39.50338...",15,NaN,201,0,1,151,"[39.494312122911424, -0.3699310988383476]",ELS ORRIOLS,0.153242,...,9.0,2,NaN,8,NaN,5,214.0,7,21.0,1


# Checking values for column 'linkid'

We note that in column linkid the most common value is 0 and is not relevant for our data, so we can drop it too.

In [10]:
# CHECKING VALUES FOR LINKID
barrios_gdf['linkid'].values


array([ 0,  0,  0, 18,  0,  0,  0,  0,  0,  0, 34,  0, 32,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 21, 15,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 24,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,
        0,  0,  0], dtype=int64)

In [11]:
barrios_gdf = barrios_gdf.drop('linkid', axis = 1)
barrios_gdf

,geometry,coddistrit,gis_gis_barrios_area,object_id_barrio,codbarrio,coddistbar,geo_point_2d,nombre_barrio,%_zona_verde,id_caract_%_zona_verde,...,num_colegios,id_caract_num_colegios,num_chargestations,id_caract_num_chargestations,pm25,id_caract_pm25,num_contenedores,id_caract_num_contenedores,num_transporte,id_caract_num_transporte
0,"POLYGON ((-0.33459 39.45478, -0.33326 39.45487...",11,917112.56250,62,5,115,"[39.44628964870906, -0.3326600366971329]",NATZARET,0.118869,3,...,6.0,2,NaN,8,10.0,5,184.0,7,11.0,1
1,"POLYGON ((-0.38124 39.45463, -0.38281 39.44951...",9,374887.53125,63,3,093,"[39.45082750748332, -0.3853982961775011]",LA CREU COBERTA,0.037566,3,...,1.0,2,2.0,8,21.0,5,162.0,7,8.0,1
2,"POLYGON ((-0.34709 39.47548, -0.34144 39.47379...",13,NaN,76,2,132,"[39.47198431553791, -0.3450427029615464]",CIUTAT JARDI,0.205871,3,...,5.0,2,NaN,8,12.0,5,238.0,7,10.0,1
3,"POLYGON ((-0.28767 39.55682, -0.28829 39.55654...",17,NaN,601,5,175,"[39.55880504953868, -0.3031699565111558]",RAFALELL-VISTABELLA,NaN,3,...,NaN,2,NaN,8,12.0,5,NaN,7,NaN,1
4,"POLYGON ((-0.33151 39.48561, -0.33254 39.48046...",13,NaN,137,5,135,"[39.48035805242704, -0.3410905938386986]",LA CARRASCA,0.187377,3,...,10.0,2,NaN,8,12.0,5,237.0,7,21.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"POLYGON ((-0.39047 39.47825, -0.38996 39.47754...",3,496668.75000,28,3,033,"[39.473565513491664, -0.39098170775459257]",LA PETXINA,0.087284,3,...,11.0,2,NaN,8,12.0,5,279.0,7,14.0,1
84,"POLYGON ((-0.37567 39.47381, -0.37423 39.47351...",1,438658.31250,37,6,016,"[39.47010643870186, -0.3760489821791233]",SANT FRANCESC,0.134529,3,...,2.0,2,2.0,8,12.0,5,580.0,7,27.0,1
85,"POLYGON ((-0.35018 39.42223, -0.35011 39.42227...",19,NaN,249,3,193,"[39.406290326215334, -0.33967854582733203]",PINEDO,NaN,3,...,2.0,2,NaN,8,10.0,5,185.0,7,14.0,1
86,"POLYGON ((-0.36720 39.50425, -0.36767 39.50338...",15,NaN,201,1,151,"[39.494312122911424, -0.3699310988383476]",ELS ORRIOLS,0.153242,3,...,9.0,2,NaN,8,NaN,5,214.0,7,21.0,1


In [28]:
barrios_gdf[['object_id_barrio','%_zona_verde']].isnull().sum()

object_id_barrio     0
%_zona_verde        12
dtype: int64